In [1]:
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook
from PIL import Image
import numpy as np
import pandas as pd
import torch
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
RESCALE_SIZE = 320
DATA_MODES = ['train', 'val', 'test']

In [3]:
# path to folders with images
folder_base = '/kaggle/input/2022-ukraine-russia-war-equipment-losses-oryx/'
folder_img_ru = ['img_russia/img_russia/', 'img_russia_2024-07-12/img_russia/']
folder_img_ua = ['img_ukraine/img_ukraine/', 'img_ukraine_2024-07-12/img_ukraine/']

# path to images metadata
path_img_metadata_ru = '/kaggle/input/2022-ukraine-russia-war-equipment-losses-oryx/img_russia_losses_metadata.csv'
path_img_metadata_ua = '/kaggle/input/2022-ukraine-russia-war-equipment-losses-oryx/img_ukraine_losses_metadata.csv'

# path to csv-files
path_losses_ru = '/kaggle/input/2022-ukraine-russia-war-equipment-losses-oryx/losses_russia.csv'
path_losses_ua = '/kaggle/input/2022-ukraine-russia-war-equipment-losses-oryx/losses_ukraine.csv'

directory_path = '/kaggle/working/model_svs/'

In [4]:
target_name = 'equipment'

In [5]:
df = pd.read_csv(path_img_metadata_ru)

In [126]:
# Drop rows where 'model' column contains the word 'Unknown' (case insensitive)
df = df[~df[target_name].str.contains('unknown', case=False, na=False)]

In [6]:
df_filtered = df[df['file'].str.contains(r'\bcapt\b', case=False, na=False)]

In [7]:
capt_df = df_filtered[df_filtered.groupby(target_name)[target_name].transform('count') > 19]

In [8]:
class_counts = capt_df[target_name].value_counts()
val_data = pd.DataFrame()
train_data = capt_df.copy()

In [10]:
class_counts

equipment
Infantry_Fighting_Vehicles                 175
Trucks,_Vehicles,_and_Jeeps                150
Tanks                                      133
Armoured_Fighting_Vehicles                  74
Engineering_Vehicles_And_Equipment          67
Reconnaissance_Unmanned_Aerial_Vehicles     59
Towed_Artillery                             24
Name: count, dtype: int64

In [240]:
for cls, count in class_counts.items():
    if count <= 5:
        continue
    elif count <= 10:
        n_samples = 2
    elif count <= 20:
        n_samples = 3
    elif count <= 100:
        n_samples = round(count * 0.1)
    else:
        n_samples = round(count * 0.1)

    # Select n_samples from the class for validation
    val_samples = train_data[train_data[target_name] == cls].sample(n_samples, random_state=42)
    
    # Append to validation set
    val_data = pd.concat([val_data, val_samples])
    
    # Remove selected instances from train set
    train_data = train_data.drop(val_samples.index)

In [241]:
print(f"Validation set size: {len(val_data)}")
print(f"Training set size: {len(train_data)}")

Validation set size: 68
Training set size: 614


In [242]:
from collections import defaultdict
from itertools import combinations

In [243]:
def custom_transform(img, rescale_size):
    image = img.resize((rescale_size, rescale_size), resample=Image.BILINEAR)
    return image

In [244]:
def preproc(dataframe):
    index_list = []
    image_list = []
    label_list = []
    for index, row in tqdm(dataframe.iterrows(), total=dataframe.shape[0]):
        if row['folder'] == 'img_russia':
            img_path = os.path.join(folder_base, folder_img_ru[0])
        else:
            img_path = os.path.join(folder_base, folder_img_ru[1])
    
        img_path = os.path.join(img_path, row['equipment'], row['file'])
    
        # Check if the corresponding label file exists
        if not os.path.exists(img_path):
            print(f"Label file missing for {img_path}, skipping.")
            continue
        
        try:
            # Open the image
            img = Image.open(img_path)
            if img.mode != 'RGB': 
                img = img.convert('RGB')
            if np.array(img).shape[2] != 3:
                print(np.array(img).shape)
            resized_img = custom_transform(img, RESCALE_SIZE)
            image_list.append(resized_img)
            label_list.append(row[target_name])
            index_list.append(index)
        
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

    return image_list, label_list, index_list

In [245]:
train_images, train_labels, train_index = preproc(train_data)

100%|██████████| 614/614 [00:11<00:00, 51.74it/s]


In [246]:
val_images, val_labels, val_index = preproc(val_data)

100%|██████████| 68/68 [00:01<00:00, 55.89it/s]


In [247]:
from sklearn.preprocessing import LabelEncoder

In [248]:
le = LabelEncoder()
le.fit(list(set(train_labels)))

LabelEncoder()

In [249]:
enc_tlabels = le.transform(train_labels)
enc_vlabels = le.transform(val_labels)

In [250]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [251]:
MEAN = [0.5097, 0.524, 0.5099]
STD = [0.212, 0.212, 0.237]

In [252]:
transform_v1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.25),
    transforms.RandomRotation(degrees=25),
    transforms.RandomPerspective(distortion_scale=0.6, p=0.25),
    transforms.Normalize(MEAN, STD)
])

In [206]:
import random

In [207]:
class TripletDataset(Dataset):
    def __init__(self, images, labels, mode, transform):
        super().__init__()
        # режим работы
        self.mode = mode
        self.transform = transform
        self.labels = labels            # np.array
        self.images = images            #

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        
        self.used_ind = {key: [] for key in np.unique(self.labels)}
        self.len_ = len(self.images)

    def __len__(self):
        return self.len_

    def __getitem__(self, index):
        img1 = self.transform(self.images[index])
        label = self.labels[index]
        
        indices = np.where(self.labels == label)[0]                   # all indices class
        if len(self.used_ind[label]) == len(indices):
            self.used_ind[label] = []
        indices = np.delete(indices, np.where(indices == index))      # delete ancor index
        if len(self.used_ind[label]) == len(indices) and index not in self.used_ind[label]:
            self.used_ind[label] = []  # !!!
        else:
            indices = np.delete(indices, np.where(np.isin(indices, self.used_ind[label])))    # delete used element as positive

        positive_idx = random.choice(indices)
        img2 = self.transform(self.images[positive_idx])
 
        self.used_ind[label].append(positive_idx)
        if self.mode == 'test':
            return img1, img2
        else:
            label = self.labels[index]
            return img1, img2, torch.tensor(label, dtype=torch.long)

In [208]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [209]:
def TripletLoss(loss_fn, embeddings, labels, mode='train', batch_size=None):
    distances = torch.cdist(embeddings, embeddings, p=2)  # Pairwise Euclidean distances
    labels = labels.unsqueeze(1)  # Shape: [N, 1]
    mask_positive = labels == labels.T  # Positive mask
    mask_negative = ~mask_positive  # Negative mask

    # Hard positives: Max distance within the positive mask
    distances_positive = distances.clone()
    distances_positive[~mask_positive] = -float('inf')  # Ignore non-positives
    hard_positive_indices = distances_positive.argmax(dim=1)
    # Hard negatives: Min distance within the negative mask
    distances_negative = distances.clone()
    distances_negative[~mask_negative] = float('inf')  # Ignore non-negatives
    hard_negative_indices = distances_negative.argmin(dim=1)

    anchor = embeddings
    positive = embeddings[hard_positive_indices]  # Hard positive embeddings
    negative = embeddings[hard_negative_indices]
    
    if mode == 'val':
        anchor = anchor[:batch_size]
        positive = positive[:batch_size]
        negative = negative[:batch_size]
    
    return loss_fn(anchor, positive, negative)

In [210]:
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score, average_precision_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

In [211]:
class Metrics():
    def __init__(self):
        self.all_actual = list()
        self.all_embeddings = []

    def batch_step(self, actualv, embeddings):
        self.all_actual.append(actualv.numpy(force=True))
        self.all_embeddings.append(embeddings.numpy(force=True))
    
    def get_metrics(self, train_emb, train_lbl, top_k=1):
        self.all_actual = np.concatenate(self.all_actual) 
        self.all_embeddings = np.concatenate(self.all_embeddings)           # collect all val data
        train_emb = train_emb.numpy(force=True)
        train_lbl = train_lbl.numpy(force=True)                             # train all data
        distance_matrix = pairwise_distances(self.all_embeddings, 
                                             np.concatenate([self.all_embeddings, train_emb]), 
                                             metric="euclidean", force_all_finite=False)  # distance matrix val embeddings to database emb. Shape N_val , N_all
        np.fill_diagonal(distance_matrix, np.inf)                 # to prevent self-selection distance between same vector=0
        ranked_indices = np.argsort(distance_matrix, axis=1)      # Sort by distance
        all_labels = np.concatenate([self.all_actual, train_lbl]) # full database
        ranked_labels = all_labels[ranked_indices]
        
        #similarity_matrix = cosine_similarity(self.all_embeddings) # For cosine similarity metric None
        y_pred = list()
        for lbl in ranked_labels:
            if top_k == 3:
                if lbl[1] == lbl[2]:
                    y_pred.append(lbl[1])
                else:
                    y_pred.append(lbl[0])
            else:
                y_pred.append(lbl[0])      
                                                    # Knn for k<=3 top sample
        ap_scores = []
        for i in range(len(self.all_actual)):
            y_true = (ranked_labels[i][:-1] == self.all_actual[i]).astype(int)  # Binary relevance (ground_truth)
            y_score = -distance_matrix[i][ranked_indices[i]][:-1]
            ap_scores.append(average_precision_score(y_true, y_score))

        y_pred = np.array(y_pred)
        mAP = np.mean(ap_scores)
        recall = recall_score(self.all_actual, y_pred, average='weighted', zero_division=0)
        precision = precision_score(self.all_actual, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(self.all_actual, y_pred, average='weighted', zero_division=0)
        accuracy = accuracy_score(self.all_actual, y_pred)
        
        return recall, precision, f1, accuracy, mAP

In [212]:
class MetricsWriter():
    def __init__(self, model, model_name='model_name', save_treshold=10):
        self.loss_val = list()
        self.loss_train = list()
        self.acc_val = list()
        self.mAP = list()
        
        self.epoch = 0
        self.start_epoch = 0

        self.save_treshold = save_treshold
        self.model_name = model_name
        self.model_type, self.module = model.get_info()
        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        self.writer = SummaryWriter('/kaggle/working/runs/' + self.model_name + '{}_{}_{}'.format(self.model_type, self.module, self.timestamp))

    def get_list_metrics(self):
        return self.loss_val, self.loss_train, self.acc_val

    def print_epoch(self, epoch):
        self.start_epoch = time.time()
        self.epoch = epoch
        print('EPOCH {}:'.format(epoch))

    def print_time(self):
        end_epoch = time.time()
        elapsed = end_epoch - self.start_epoch
        print("Time per epoch {}s".format(elapsed))

    def get_best_epoch_by_map(self):
        return self.mAP.index(max(self.mAP))

    def writer_step(self, loss, vloss, recall, precision, f1, acc, mAP):
        print('LOSS train {} valid {}'.format(loss, vloss))
        print('Accuracy valid {}'.format(acc))
        print('Recall valid {}'.format(recall))
        print('Precision valid {}'.format(precision))
        print('Val F1->{}'.format(f1))
        print('Val mAP->{}'.format(mAP))
        
        self.loss_train.append(loss)
        self.loss_val.append(vloss)
        self.acc_val.append(acc)
        self.mAP.append(mAP)

        self.writer.add_scalars('Training vs. Validation Loss',
                    { 'Training': loss, 'Validation': vloss },
                    self.epoch)
        self.writer.add_scalars('Validation Metrics',
                    { 'Validation Recall': recall, 'Validation Precision': precision, 'Validation F1': f1
                    }, self.epoch)
        self.writer.add_scalars('Validation Accuracy',
                    { 'Validation Accuracy': acc
                    }, self.epoch)
        self.writer.add_scalars('Validation mAP',
                    { 'Validation mAP': mAP
                    }, self.epoch)

    def save_model(self, model):
        if (self.epoch) % self.save_treshold == 0:
            model_path = '/kaggle/working/model_svs/' + self.model_name + '_{}_{}_{}_{}'.format(self.model_type, 
                                                                                                self.module, self.timestamp, 
                                                                                                self.epoch)
            torch.save(model.state_dict(), model_path)

In [213]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn import init
import math

In [214]:
class SpatialGroupEnhance(nn.Module):
    def __init__(self, groups = 64):
        super(SpatialGroupEnhance, self).__init__()
        self.groups = groups
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.weight = Parameter(torch.zeros(1, groups, 1, 1))
        self.bias = Parameter(torch.ones(1, groups, 1, 1))
        self.sig = nn.Sigmoid()

    def forward(self, x): # (b, c, h, w)
        b, c, h, w = x.size()
        x = x.view(b * self.groups, -1, h, w) 
        xn = x * self.avg_pool(x)
        xn = xn.sum(dim=1, keepdim=True)
        t = xn.view(b * self.groups, -1)
        t = t - t.mean(dim=1, keepdim=True)
        std = t.std(dim=1, keepdim=True) + 1e-5
        t = t / std
        t = t.view(b, self.groups, h, w)
        t = t * self.weight + self.bias
        t = t.view(b * self.groups, 1, h, w)
        x = x * self.sig(t)
        x = x.view(b, c, h, w)
        return x

In [215]:
class ResNet18WithSGE(nn.Module):
    def __init__(self, num_classes, groups, pretrained=True):
        super(ResNet18WithSGE, self).__init__()
        
        self.resnet = models.resnet18(pretrained=pretrained)
        self.layer0 = nn.Sequential(
            self.resnet.conv1,
            self.resnet.bn1,
            self.resnet.relu,
            self.resnet.maxpool
        )
        self.layer1 = self.resnet.layer1
        self.layer2 = self.resnet.layer2
        self.layer3 = self.resnet.layer3
        self.layer4 = self.resnet.layer4
        self.avgpool = self.resnet.avgpool
        self.in_features = self.resnet.fc.in_features  # Get the number of input features for the original FC layer
        self.fc = nn.Linear(self.in_features, num_classes)  # New FC layer
        
        self.module1 = SpatialGroupEnhance(groups[0])
        self.module2 = SpatialGroupEnhance(groups[1])
        self.module3 = SpatialGroupEnhance(groups[2])
        self.module4 = SpatialGroupEnhance(groups[3])
        nn.init.xavier_uniform_(self.fc.weight)


    def get_info(self):
        return 'ResNet18', 'SGE'
    

    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.module1(x)
        x = self.layer2(x)
        x = self.module2(x)
        x = self.layer3(x)
        x = self.module3(x)
        x = self.layer4(x)
        x = self.module4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [216]:
class ResNet18WithSGEFeatureExtractor(ResNet18WithSGE):
    def __init__(self, num_classes, groups, pretrained=True):
        super(ResNet18WithSGEFeatureExtractor, self).__init__(num_classes, groups, pretrained)

    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.module1(x)
        x = self.layer2(x)
        x = self.module2(x)
        x = self.layer3(x)
        x = self.module3(x)
        x = self.layer4(x)
        x = self.module4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)  # Flatten before FC layer (Feature Vector)
        return x 

In [217]:
import time
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

In [218]:
BATCH_SIZE = 64

In [254]:
train_dataset = TripletDataset(train_images, enc_tlabels, 'train', transform_v1)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)
val_dataset = TripletDataset(val_images, enc_vlabels, 'val', transform_v1)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=4)

In [265]:
def train_step(model, loss_fn, opt, loader, batch_size):
    loss_per_batches = 0
    elapsed = 0
    start_epoch2 = time.time()
    all_embeddings = list()
    all_labels = list()
    for i, data in tqdm(enumerate(loader), total=len(train_labels)//(batch_size)):

        start_epoch = time.time()
        img1, img2, labels = data
        img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
        features = torch.cat((img1, img2), dim=0)
        labels = torch.cat((labels, labels), dim=0)
        # concat img & label
        opt.zero_grad()

        y_pred = model(features)
        loss = TripletLoss(loss_func, y_pred, labels, batch_size=batch_size)
        loss.backward()

        opt.step()

        loss_per_batches += loss
        end_epoch = time.time()
        elapsed += (end_epoch - start_epoch)
        all_embeddings.append(y_pred[:batch_size])
        all_labels.append(labels[:batch_size])

    print("train = " + str(elapsed))
    print("train + load = " + str(time.time() - start_epoch2))
    return loss_per_batches/(i+1), torch.cat(all_embeddings, dim=0), torch.cat(all_labels, dim=0)

In [65]:
directory_path = '/kaggle/working/model_svs/'

In [66]:
if not os.path.exists(directory_path):
    # Create the directory
    os.makedirs(directory_path)
    print(f"Directory created at: {directory_path}")
else:
    print(f"Directory already exists at: {directory_path}")

Directory created at: /kaggle/working/model_svs/


In [266]:
def train(model, mw, loss_func, opt, train_loader, val_loader, batch_size, epochs=50):
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, epochs)

    for epoch in range(epochs):
        mw.print_epoch(epoch + 1)
        metrics_valid = Metrics()

        model.train()
        avg_loss, train_emb, train_labels = train_step(model, loss_func, opt, 
                                                       train_loader, batch_size)
        model.eval()

        vloss = 0
        counter = 0
        vclassloss = 0
        with torch.inference_mode():
            for i, vdata in enumerate(val_loader):
                vfeatures1, vfeatures2, vlabels = vdata
                vfeatures1, vfeatures2, vlabels = vfeatures1.to(device), vfeatures2.to(device), vlabels.to(device)
                vfeatures = torch.cat((vfeatures1, vfeatures2), dim=0)
                vlabels = torch.cat((vlabels, vlabels), dim=0)
                
                outputs = model(vfeatures)
                metrics_valid.batch_step(vlabels[:batch_size], outputs[:batch_size])

                bloss = TripletLoss(loss_func, outputs, vlabels, mode='val', batch_size=batch_size)
                vloss += bloss
                counter = i

        avg_vloss = vloss / (counter + 1)
        scheduler.step()

                    # concat dataset
        valrecall, valprecision, valf1, valacc, mAP = metrics_valid.get_metrics(train_emb, train_labels)
        mw.writer_step(avg_loss, avg_vloss, valrecall, valprecision, valf1, valacc, mAP)

        mw.save_model(model)

        mw.print_time()

In [291]:
model = ResNet18WithSGEFeatureExtractor(num_classes=11, 
                                        groups=[2, 4, 4, 4], pretrained=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [292]:
checkpoint_path = '/kaggle/input/model-weights/second_model_ResNet18_SGE_20250112_190440_20'
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint)

<ipython-input-292-f3187fd25720>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


<All keys matched successfully>

In [293]:
mw = MetricsWriter(model, model_name='capt20_1-5_1_try_model', save_treshold=10)

In [294]:
loss_func = nn.TripletMarginWithDistanceLoss(margin=0.5)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [295]:
train(model, mw, loss_func, optimizer, train_dataloader, val_dataloader, BATCH_SIZE, epochs=100)

EPOCH 1:


10it [00:07,  1.36it/s]                      

train = 3.369797945022583
train + load = 7.628100156784058


LOSS train 7.370855808258057 valid 4.51847505569458
Accuracy valid 0.3888888888888889
Recall valid 0.3888888888888889
Precision valid 0.38770893458393463
Val F1->0.38507054255393913
Val mAP->0.22715243414137604
Time per epoch 9.510800838470459s
EPOCH 2:


10it [00:06,  1.45it/s]                      

...

10it [00:06,  1.50it/s]                      

train = 3.4544503688812256
train + load = 6.863775253295898


LOSS train 0.5243862271308899 valid 0.7548172473907471
Accuracy valid 0.7083333333333334
Recall valid 0.7083333333333334
Precision valid 0.711686056998557
Val F1->0.6999462968997019
Val mAP->0.4994501353692381
Time per epoch 8.813463687896729s
EPOCH 100:


10it [00:06,  1.47it/s]                      


train = 3.4478232860565186
train + load = 7.0370800495147705
LOSS train 0.527546763420105 valid 0.9187754392623901
Accuracy valid 0.6527777777777778
Recall valid 0.6527777777777778
Precision valid 0.6481068121693121
Val F1->0.6424457093394802
Val mAP->0.5207091667322279
Time per epoch 9.06659197807312s
